In [1]:
import os
import re
from tqdm import tqdm
import random
from scipy.io import wavfile
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten 
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import python_speech_features as psf
print(tf.__version__)

2.2.0


In [14]:
def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window = int(rate/10), min_periods = 1, center = True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

def read_data(path):

    signal, rate = librosa.load(path, sr=16000)
    mask = envelope(signal, rate, 0.005)#envelope to filter out silence
    signal_masked = signal[mask]
    length =len(signal_masked)/rate
    
    return path, signal_masked,length


def build_feat_list(f):
    X = []
    _min, _max = float('inf'), -float('inf')
    step = 2
    signal = df.at[f,'data']
    rate = 16000
    step_rate = rate*step
    
    if step < df.at[f, 'length']:
        
        #rand_index = np.random.randint(0, signal.shape[0]-step_rate)
        #sample = signal[rand_index : rand_index+step_rate]
        
        #X_sample = librosa.feature.mfcc(y =sample, sr = rate, n_mfcc=26)
        #_min = min(np.amin(X_sample), _min)
        #_max = max(np.amax(X_sample), _max)
        #X.append(X_sample)
        for i in range(int(df.at[f, 'length']/step)):
            sample = signal[ i*rate*step : int((i+1)*step*rate)]
            sample = np.array(sample)
            #convert data
            X_sample = librosa.feature.mfcc(y =sample, sr = rate, n_mfcc=26)
            #X_sample = psf.mfcc(sample, rate, numcep = 64, nfilt=128, nfft=1103).T
            #print(sample)
            _min = min(np.amin(X_sample), _min)
            _max = max(np.amax(X_sample), _max)
            X.append(X_sample)
    else:
        sample = np.pad(signal,(0,step_rate - len(signal)), 'constant', constant_values=0)
        X_sample = librosa.feature.mfcc(y =sample, sr = rate, n_mfcc=26)
        _min = min(np.amin(X_sample), _min)
        _max = max(np.amax(X_sample), _max)
        X.append(X_sample)
        
    X = np.array(X)
    X = (X - _min)/(_max - _min)
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

    return X 

def predict(X):
    score = np.sum(loaded_model.predict(X), axis=0)
    label = np.argmax(score)
    
    return label



In [17]:
fname = []
data = []
length = []

for f in tqdm(os.listdir('data/test')):
    a, d, l= read_data('data/test'+'/'+f)
    fname.append(f)
    data.append(d)
    length.append(l)
dict = {'id': fname, 'data': data, 'length': length}
df = pd.DataFrame(dict)
df

100%|██████████| 228/228 [00:28<00:00,  7.87it/s]


,id,data,length
0,test_128.wav,"[-0.010009766, -0.0073547363, -0.0014038086, 0...",13.542438
1,test_97.wav,"[0.003479004, 0.0009460449, 0.0027160645, -0.0...",8.718500
2,test_125.wav,"[0.0, 0.0, -0.00024414062, 0.0017089844, -0.00...",14.523438
3,test_84.wav,"[-0.0032043457, -0.00390625, -0.0062561035, -0...",17.114062
4,test_204.wav,"[-0.03652954, 0.04626465, -0.023620605, -0.104...",12.305938
...,...,...,...
223,test_220.wav,"[-0.011871338, -0.011260986, -0.009155273, -0....",15.594750
224,test_186.wav,"[-0.13266818, -0.34102347, -0.07698086, 0.1503...",15.894875
225,test_200.wav,"[-0.0072021484, -0.003967285, -0.0014038086, -...",10.853687
226,test_199.wav,"[0.002166748, 0.00033569336, 0.001739502, -0.0...",3.028938


In [30]:
print(df.at[15, 'data'].shape[0]/16000)

2.1705


In [7]:
loaded_model = tf.keras.models.load_model('/tmp/model2')
X = build_feat_list(10)
print(X.shape)

(1, 26, 63, 1)


In [8]:


#loaded_model = tf.keras.models.load_model('/tmp/model1')
label = []
print('model loaded')
for f in tqdm(df.index):
    X = build_feat_list(f)
    l = predict(X)
    label.append(l) 
print(label)
#df['label']=label

#save csv

  1%|          | 2/228 [00:00<00:12, 18.54it/s]

model loaded


100%|██████████| 228/228 [00:04<00:00, 48.69it/s]

[2, 1, 1, 5, 2, 2, 1, 3, 3, 5, 5, 0, 3, 1, 3, 3, 0, 5, 4, 5, 5, 2, 5, 2, 5, 3, 5, 1, 5, 2, 2, 3, 5, 5, 2, 1, 3, 2, 5, 1, 3, 3, 5, 1, 5, 1, 1, 5, 1, 5, 1, 0, 5, 0, 3, 2, 5, 1, 2, 4, 1, 5, 5, 3, 5, 2, 1, 0, 5, 2, 5, 5, 2, 0, 1, 4, 5, 0, 1, 1, 3, 1, 5, 5, 5, 2, 2, 1, 5, 1, 3, 1, 5, 2, 3, 3, 5, 5, 3, 5, 5, 3, 2, 1, 1, 1, 5, 1, 0, 5, 1, 2, 4, 0, 2, 1, 3, 5, 5, 2, 1, 0, 0, 3, 2, 2, 5, 2, 5, 2, 3, 1, 1, 5, 5, 5, 1, 4, 4, 5, 0, 3, 1, 2, 5, 5, 3, 3, 5, 5, 3, 2, 5, 5, 5, 5, 0, 4, 5, 5, 4, 5, 5, 3, 3, 5, 2, 0, 5, 5, 2, 5, 5, 2, 2, 2, 1, 1, 3, 1, 1, 5, 5, 5, 5, 5, 3, 5, 3, 1, 2, 5, 3, 0, 2, 5, 2, 5, 1, 3, 3, 2, 5, 5, 2, 1, 5, 3, 3, 2, 3, 5, 4, 5, 5, 5, 5, 5, 1, 5, 2, 5, 2, 1, 2, 4, 3, 2]


In [15]:
label = []
print('model loaded')
for f in tqdm(df.index):
    X = build_feat_list(f)
    l = predict(X)
    label.append(l) 
print(label)

  1%|          | 2/228 [00:00<00:16, 13.90it/s]

model loaded


100%|██████████| 228/228 [00:08<00:00, 27.28it/s]

[2, 5, 2, 5, 5, 5, 5, 5, 1, 2, 3, 2, 5, 5, 5, 1, 1, 5, 1, 5, 5, 5, 5, 1, 5, 2, 3, 2, 3, 5, 5, 3, 5, 5, 2, 3, 5, 1, 5, 2, 1, 5, 5, 3, 3, 5, 1, 1, 5, 2, 2, 2, 5, 5, 5, 1, 5, 1, 5, 3, 5, 3, 3, 1, 0, 5, 3, 5, 5, 1, 5, 5, 1, 2, 2, 5, 2, 5, 5, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, 1, 0, 5, 5, 5, 0, 1, 5, 5, 5, 5, 1, 1, 5, 3, 5, 5, 5, 5, 5, 2, 1, 5, 5, 5, 3, 5, 5, 5, 1, 2, 5, 1, 5, 5, 4, 5, 1, 3, 1, 5, 1, 1, 5, 2, 0, 2, 5, 1, 3, 5, 2, 5, 5, 1, 5, 1, 3, 5, 5, 1, 5, 5, 3, 2, 1, 1, 2, 1, 5, 3, 5, 5, 5, 5, 5, 1, 2, 3, 5, 5, 5, 5, 3, 1, 5, 3, 3, 5, 5, 5, 3, 5, 1, 1, 1, 3, 0, 5, 0, 5, 5, 5, 1, 3, 2, 5, 5, 3, 3, 5, 0, 0, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 2, 5, 5, 1, 5, 5, 1, 5, 3, 1, 5, 3, 5, 1]


In [18]:
n = []
for i in df.index:
    x = re.split("_", df.at[i,'id'])
    number = re.split("\.", x[1])
    n.append(int(number[0]))
print(n)

[128, 97, 125, 84, 204, 67, 8, 55, 205, 56, 143, 188, 222, 78, 105, 225, 202, 179, 120, 146, 54, 132, 134, 201, 113, 158, 31, 130, 193, 34, 166, 69, 33, 107, 122, 51, 148, 119, 124, 57, 156, 173, 95, 16, 189, 37, 116, 30, 216, 131, 19, 207, 172, 136, 151, 22, 219, 108, 32, 12, 9, 170, 142, 66, 138, 137, 117, 209, 65, 217, 61, 100, 76, 210, 72, 115, 87, 64, 227, 11, 86, 111, 194, 48, 7, 183, 214, 223, 60, 213, 104, 43, 191, 82, 147, 180, 41, 226, 140, 18, 75, 50, 141, 133, 40, 139, 149, 47, 190, 129, 109, 112, 184, 187, 29, 27, 20, 153, 127, 196, 62, 203, 167, 49, 77, 83, 106, 91, 90, 99, 174, 68, 103, 89, 2, 160, 198, 126, 157, 211, 163, 96, 94, 102, 192, 150, 59, 36, 93, 81, 182, 63, 145, 24, 197, 164, 161, 110, 74, 1, 45, 114, 6, 80, 42, 13, 159, 71, 224, 175, 38, 88, 70, 52, 181, 25, 218, 185, 178, 221, 154, 14, 152, 26, 5, 135, 10, 85, 28, 155, 73, 176, 79, 92, 17, 162, 121, 21, 53, 168, 177, 0, 101, 171, 15, 35, 3, 44, 195, 123, 206, 215, 23, 212, 165, 118, 208, 4, 169, 58, 98, 14

In [19]:
classes = ['awake', 'diaper', 'hug', 'hungry', 'sleepy', 'uncomfortable']
label_txt = []
for i in range(len(label)):
    label_txt.append(classes[label[i]])
df['label'] = label_txt
df['n'] = n
df.n = df.n.astype(int)

df.sort_values("n", axis = 0, ascending = True, 
                inplace = True, na_position ='last') 

df
#final = df[['id','label']]
#final
#final

,id,data,length,label,n
201,test_0.wav,"[0.03387451, 0.046325684, 0.038879395, 0.03518...",14.355562,uncomfortable,0
159,test_1.wav,"[-0.059391413, -0.04179137, 0.05396723, 0.0777...",14.396812,diaper,1
134,test_2.wav,"[0.062347412, -0.0037841797, 0.081604004, 0.17...",19.649250,uncomfortable,2
206,test_3.wav,"[0.0004272461, 0.0010375977, 0.00012207031, -0...",10.953125,uncomfortable,3
217,test_4.wav,"[0.052825928, -0.023925781, -0.07220459, -0.11...",14.874688,diaper,4
...,...,...,...,...,...
87,test_223.wav,"[-0.003326416, -0.0034179688, -0.0036315918, -...",5.205188,uncomfortable,223
168,test_224.wav,"[-0.13461304, -0.14828491, -0.1605835, -0.1502...",11.699500,hug,224
15,test_225.wav,"[0.001159668, 0.0010681152, -0.0008239746, -0....",2.170500,diaper,225
97,test_226.wav,"[0.0028076172, 0.002746582, 0.0033569336, 0.00...",14.996688,diaper,226


In [20]:
final = df[['id','label']]
final

,id,label
201,test_0.wav,uncomfortable
159,test_1.wav,diaper
134,test_2.wav,uncomfortable
206,test_3.wav,uncomfortable
217,test_4.wav,diaper
...,...,...
87,test_223.wav,uncomfortable
168,test_224.wav,hug
15,test_225.wav,diaper
97,test_226.wav,diaper


In [21]:
final.to_csv('Result7.csv',index=0)